In [47]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from urllib import parse
import os
from datetime import datetime
import datetime

In [48]:
code_df = pd.read_csv('code_a.csv', encoding='cp949')
code_df

,회사명,종목코드
0,GS글로벌,1250
1,HSD엔진,82740
2,KG케미칼,1390
3,LG이노텍,11070
4,OCI,10060
5,SK네트웍스,1740
6,SK이노베이션,96770
7,STX,11810
8,WISCOM,24070
9,갤럭시아에스엠,11420


In [49]:
def price(code):
#     code_n = code(name)
    price_list = []
    url = 'https://finance.naver.com/item/sise_day.nhn'
    for num in range(1,30):
        params = {
            'code' : code,
            'page' : '{}'.format(num)
        }
        resp = requests.get(url, params = params)
        soup = BeautifulSoup(resp.content, 'html.parser')
        for tr in soup.find_all('tr')[2:14]:
            new_dict = {}
            td_list = tr.find_all('td')
            if not td_list[0].text.strip():
                continue
            tdtext_list = ['날짜', '종가', '전일비', '시가', '고가', '저가', '거래량']
            for i in range(7):
                new_dict[tdtext_list[i]] = td_list[i].text.strip()
            price_list.append(new_dict)
            df_price = pd.DataFrame(price_list, columns = ['날짜', '종가', '전일비', '시가', '고가', '저가', '거래량'])
    return df_price

In [50]:
def crawling(code): #code = 종목코드
    df_crawling = pd.DataFrame()
    
    
    ###
    temp_page_url = 'https://finance.naver.com/item/news_news.nhn?code={}&page={}&sm=title_entity_id.basic&clusterId='.format(code, 1) 
    #request
    
    lastPageResp = requests.get(temp_page_url)
    lastPagehtml = lastPageResp.text
    
    #soup초기화
    soup = BeautifulSoup(lastPagehtml)
    
    href = soup.find('td', class_='pgRR').a['href']
    url = parse.urlparse(href)
    
    #Http URL Parameter를 다루려면 parse_qs() 메서드를 사용
    #parse_qs() 는 지정된 쿼리스트링을 해석하여 dict() 로 반환
    url_query = parse.parse_qs(url.query)
    
    pg = url_query['page'][0]
        
    ##400page가 넘는 경우 반복되는 news만 나오므로 400 = pg
    if int(pg) >= int(401):
        pg = int(400)
    else:
        pg = pg
    
    ###      

    for page in range(1,int(pg)+1):
        #종목에서 더 많은 페이지를 설정하는 경우, 마지막페이지만 계속 반복
        page_url = 'https://finance.naver.com/item/news_news.nhn?code={}&page={}&sm=title_entity_id.basic&clusterId='.format(code,page)
        resp = requests.get(page_url)
        html = resp.text
        soup = BeautifulSoup(html)
        
        a_tags = soup.select('a.tit')
        title_text=[]    
        link_text=[]
        for a_tag in a_tags:
            title_text.append(a_tag.text)
            link_text.append(a_tag['href'])

        td_tags = soup.select('td.date')   
        date_text=[]             
        for td_tag in td_tags:
            date_text.append(td_tag.string)  #.string하니까 시간만 정제됨!!!

        test_dict={}
        test_dict['date']=date_text
        test_dict['title']=title_text  #title이라는 키값을 가지는 title_text 리스트의 요소들은  dict로 전환
        test_dict['link']=link_text
    
        df_crawling = df_crawling.append(pd.DataFrame(test_dict), ignore_index=True)
    return df_crawling

In [57]:
def report_crawling(new_code):
    
    period_year = 1      #오늘로부터 기간(year) 
    end_day = str(datetime.date.today())[:10]
    start_day = str(int(end_day[:4])-period_year)+end_day[4:]
    domain_url = 'https://www.wisereport.co.kr/wiseReport/reports/company.aspx'
    cookies = {
        'ASP.NET_SessionId': 'j0teti3jiziaqs1tqerkvv4q'
    }

    page_num=0
    report_list = []
    for i in range(1,10000):
        form_data = {
            'startDT': start_day,
            'endDt': end_day,
            'langTyp': 1,
            'searchTyp': None,
            'searchVal': None,
            'reVal': 'and a.cmp_cd = |{}|'.format(new_code),
            'perPage': 25,
            'curPage': i,
            'sortCol': 'anl_dt',
            'sortTyp': 'd',
            'flag': None,
            'flashYN': 1,
            'userId': 'gaps5_000543'
        }
        resp = requests.post(domain_url, cookies = cookies, data = form_data)
        soup = BeautifulSoup(resp.content,'html.parser')
        tg_contents = soup.find('tbody')

        if soup.find('tbody').find('tr').find('td').attrs['class'] == ['lst_center', 'end']:
            break
        else:
            page_num+=1
#             print('{}페이지를 가져오겠습니다.'.format(page_num))
    
        new_list=[]
        for content in tg_contents.contents[:-2]:
        
            new_dict = {}

            new_dict['title'] = content.find_all('td')[1].find('div').find('a').text

            new_dict['time']  = content.find('td').text

            summary_tag = content.find_all('td')[1].attrs['data-content']
            summary_soup = BeautifulSoup(summary_tag)
            summary_chunk = summary_soup.text
            summary_list = summary_chunk.split('▶')[1:]
            for i in range(len(summary_list)-1):
                summary_list[i] = summary_list[i][:-2]
            new_dict['summary'] = summary_list

            new_list.append(new_dict)

        report_list.extend(new_list)
        
    report_list = pd.DataFrame(report_list)
    return report_list

In [58]:
def func(name):
    new_name = name.upper()
    a = code_df[code_df['회사명'].isin([new_name])]
# name은 인풋 출력함수이므로 그 자체로 str, 별도의 ' 를 사용할 필요가 없다.
# 회사명을 정확하게 알 수 없는 경우 입력값을 포함하는 리스트를 출력하고, 다시 입력할 수 있도록 함.
    while True:
        if a.size == 0:
            print('정확한 회사명을 입력하세요')
            # 특정 문자열이 포함된 리스트 뽑기
            compony_ex = code_df[code_df.회사명.str.contains(new_name) == True]
            print(compony_ex)
            name = input('회사이름을 적어주세요 : ')
            new_name = name.upper()
            a = code_df[code_df['회사명'].isin([new_name])]
        else:
            code = a['종목코드'].values[0]
            new_code = '%06d' %code
            break

    # 함수를 사용하여 각각의 데이터프레임 출력하기
    df_news = crawling(new_code)
    df_price = price(new_code)
#     df_report = report_crawling(new_code)
    # 저장할 폴더 생성
    BASE_DIR = os.getcwd()
    SAVE_DB_DIR = os.path.join(BASE_DIR, name)
    
    if not os.path.exists(SAVE_DB_DIR):
        os.makedirs(SAVE_DB_DIR)
    
    #텍스트 파일 만들기
    temp_name = f"{name}_{new_code}_{str(datetime.date.today())}.xlsx"
    txt_name = os.path.join(SAVE_DB_DIR, temp_name)
    excel_writer = pd.ExcelWriter(txt_name, engine = 'xlsxwriter')
    df_report.to_excel(excel_writer, sheet_name='report')
    df_news.to_excel(excel_writer, sheet_name='news')
    df_price.to_excel(excel_writer, sheet_name='price')
    excel_writer.save()
#     df_price.to_excel(txt_name, sheet_name='price')
    print("저장 완료. 프로그램을 종료합니다.")

In [56]:
func('삼성화재해상보험')

저장 완료. 프로그램을 종료합니다.
